# Vector Store Generating
## Downloading the dependencies

In [2]:
%%bash 
pip install -qU langchain-nomic
pip install -U langchain-chroma
pip install langchain-openai
pip install --upgrade --quiet langchain-community gpt4all
pip install --upgrade --quiet  gpt4all > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorrt-llm 0.10.0 requires nvidia-modelopt<0.12,~=0.11, but you have nvidia-modelopt 0.0.0 which is incompatible.
tensorrt-llm 0.10.0 requires pynvml>=11.5.0, but you have pynvml 11.4.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 979.4 kB/s eta 0:00:00 0:00:01
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 139.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 51.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 121.2 MB/s eta 0:00:00
   ━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.2.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.2.2 which is incompatible.
cudf 24.2.0 requires protobuf<5,>=4.21, but you have protobuf 5.29.2 which is incompatible.
cugraph 24.2.0 requires dask-cuda==24.2.*, but you have dask-cuda 24.6.0 which is incompatible.
cugraph 24.2.0 requires rapids-dask-dependency==24.2.*, but you have rapids-dask-dependency 24.6.0 which is incompatible.
cugraph-service-server 24.2.0 requires dask-cuda==24.2.*, but you have dask-cuda 24.6.0 which is incompatible.
cugraph-service-server 24.2.0 requires rapids-dask-dependency==24.2.*, but you have rapids-dask-dependency 24.6.0 which is incompatible.
cuml 24.2.0 requires dask-cuda==24.2.*, but you have dask-cuda 24.6.0 which is incompatible.
cuml 24.2.0 requires rapids-dask-dependency==24.2.*, but you have rapids-dask-de


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


# Import Dependency

In [3]:
from langchain_community.embeddings import GPT4AllEmbeddings

In [4]:
gpt4all_embd = GPT4AllEmbeddings()

Downloading: 100%|██████████| 45.9M/45.9M [00:01<00:00, 28.4MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 543MiB/s]
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


# 讀回Cleaned data

In [1]:
import json
import xml.etree.ElementTree as ET
from langchain.docstore.document import Document  # 若尚未安裝langchain請先安裝: pip install langchain

# 從 updated_drugs.json 讀取更新後的 XML 資料 (list of XML strings)
with open("updated_drugs.json", "r", encoding="utf-8") as f:
    updated_drugs = json.load(f)

# 將 JSON 中的資料重新轉回 splitted_drugs 和 full_drugs_names
splitted_drugs = []
full_drugs_names = []

for xml_str in updated_drugs:
    root = ET.fromstring(xml_str)
    # 假設 drug-name 是在根標籤中存放 (在前面 add_drug_name_to_tags 函式中為每個元素加入，但根通常也會有)
    drug_name = root.get('drug-name', 'Unknown')

    splitted_drugs.append(xml_str)
    full_drugs_names.append(drug_name)


## Nomic Embedding

In [ ]:
# import getpass

# os.environ["NOMIC_API_KEY"] = getpass.getpass()
# #
# from langchain_nomic import NomicEmbeddings


# ## Nomic Embedding
# embeddings = NomicEmbeddings(
#     model="nomic-embed-text-v1.5",
#     # dimensionality=256,
#     # Nomic's `nomic-embed-text-v1.5` model was [trained with Matryoshka learning](https://blog.nomic.ai/posts/nomic-embed-matryoshka)
#     # to enable variable-length embeddings with a single model.
#     # This means that you can specify the dimensionality of the embeddings at inference time.
#     # The model supports dimensionality from 64 to 768.
#     # inference_mode="remote",
#     # One of `remote`, `local` (Embed4All), or `dynamic` (automatic). Defaults to `remote`.
#     # api_key=... , # if using remote inference,
#     device="cpu"
#     # The device to use for local embeddings. Choices include
#     # `cpu`, `gpu`, `nvidia`, `amd`, or a specific device name. See
#     # the docstring for `GPT4All.__init__` for more info. Typically
#     # defaults to CPU. Do not use on macOS.
# )


# doc_result = gpt4all_embd.embed_documents()


In [ ]:
# import torch.nn.functional as F
# from sentence_transformers import SentenceTransformer

# matryoshka_dim = 512

# model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
# sentences = splitted_drug_no_int[0:2]
# embeddings = model.encode(sentences, convert_to_tensor=True)
# embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
# embeddings = embeddings[:, :matryoshka_dim]
# embeddings = F.normalize(embeddings, p=2, dim=1)
# print(embeddings)

# print(len(embeddings[0]))

In [ ]:
# from sentence_transformers import SentenceTransformer

# # 載入預訓練的 Hugging Face 模型，例如 "all-MiniLM-L6-v2"
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# # 將文本轉為 Embeddings
# texts = ["Hello world", "This is a test"]
# embeddings = model.encode(texts)  # embeddings為 numpy array

# from langchain.vectorstores import Chroma

# # texts, metadata 與 embeddings 必須對應。
# # 如果您已有 embeddings 與對應的文本 (texts) 與 meta 資訊 (metadatas)
# # 假設 embeddings 為 2D numpy array，shape: (num_texts, embedding_dim)

# vectorstore = Chroma(
#     collection_name="my_collection",
#     embedding_function=None,  # 因為我們已自己算好 embeddings，所以這裡可為 None
#     persist_directory="path_to_store",
# )

# # 新增文件與其 embeddings
# vectorstore.add_texts(
#     texts=texts,
#     metadatas=[{"source": "some_source"} for _ in texts],
#     embeddings=embeddings  # 將預先計算好的 embeddings 傳入
# )

# # 接著即可使用 vectorstore 的相似度查詢等功能
# query = "Hello"
# # 將 query 先自行 embedding，再用 vectorstore 查詢
# query_emb = model.encode([query])
# docs = vectorstore.similarity_search_by_vector(query_emb[0], k=2)
# print(docs)


## GPT4All Embedding

In [6]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings

model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs,
)


GPT4AllEmbeddings(model_name='all-MiniLM-L6-v2.gguf2.f16.gguf', n_threads=None, device='cpu', gpt4all_kwargs={'allow_download': 'True'}, client=<gpt4all.gpt4all.Embed4All object at 0x7f64945a2920>)

### GPT4All with Splitting the information of single Drug

# Data vectorizing embedding


In [7]:
!pip install -U jq

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## 讀入需要使用的藥物
- drug_docs_list: 惠馨選出需要使用的藥物全數資料

In [8]:
import csv
def read_drug_list_from_csv(file_path):
    try:
        with open(file_path, mode='r', encoding='utf-8') as f:
            reader = csv.reader(f)
            drug_list = [row[0] for row in reader if row]  # 假設每行第一列是藥物名稱
        return drug_list
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return []

drug_list_csv_path = "./dataset/hackathon_drug.csv"
drug_list = read_drug_list_from_csv(drug_list_csv_path)

# 緊急處理：藥物tag塞入藥物名稱

In [9]:
import re
import json

# file_path='./up.json'
# # Open and read the JSON file
# with open(file_path, 'r') as file:
#     data = json.load(file)


## 篩選藥物
### 找出不在 drug_list 的索引
in_drug_list_indices = [index for index, drug in enumerate(full_drugs_names) if drug in drug_list]
# sel_data = []
# for index in in_drug_list_indices:
#     sel_data.append(data[index])

# ## 正則表示法刪除drug-interactions
# drug_interactions = r'<drug-interactions.*?<\/drug-interactions.*?>'

# # 先移除Drug-Interactions
# sel_data_noInt = []

# for s_d in sel_data:

#     drug_no_int = re.sub(drug_interactions, '', str(s_d), flags=re.DOTALL)
#     sel_data_noInt.append(str(drug_no_int))

# 選擇的藥物名稱
sel_drugs_names = []
for index in in_drug_list_indices:
    sel_drugs_names.append(full_drugs_names[index])
    

## 藥物轉json

In [153]:
# import xml.etree.ElementTree as ET
# import json

# def xml_to_dict(element):
#     """
#     將 XML 元素轉換為字典，鍵包含標籤名稱及其屬性。
    
#     範例：
#     <tag attr1="value1" attr2="value2">Content</tag>
#     將轉換為：
#     {'tag attr1="value1" attr2="value2"': 'Content'}
#     """
#     # 構建鍵，包含標籤名稱和屬性
#     if element.attrib:
#         attrs = ' '.join([f'{k}="{v}"' for k, v in element.attrib.items()])
#         key = f"{element.tag} {attrs}"
#     else:
#         key = element.tag

#     # 如果元素有子元素，遞歸處理
#     if list(element):
#         children = {}
#         for child in element:
#             child_dict = xml_to_dict(child)
#             child_key = list(child_dict.keys())[0]
#             child_value = child_dict[child_key]
#             if child_key in children:
#                 # 如果鍵已存在，將值轉換為列表以包含多個子元素
#                 if isinstance(children[child_key], list):
#                     children[child_key].append(child_value)
#                 else:
#                     children[child_key] = [children[child_key], child_value]
#             else:
#                 children[child_key] = child_value
#         return {key: children}
#     else:
#         return {key: element.text}

# def convert_drugs_to_json(splitted_drugs, full_drugs_names, output_file="drugs.json"):
#     all_drugs_data = []

#     for drug_xml, drug_name in zip(splitted_drugs, full_drugs_names):
#         try:
#             root = ET.fromstring(drug_xml)
#             drug_dict = xml_to_dict(root)
#             # 可在此放入藥物名稱等metadata
#             drug_dict["drug_name"] = drug_name
#             all_drugs_data.append(drug_dict)
#         except Exception as e:
#             print(f"Error processing drug {drug_name}: {e}")

#     # 將結果寫出 JSON 檔案
#     with open(output_file, "w", encoding="utf-8") as f:
#         json.dump(all_drugs_data, f, ensure_ascii=False, indent=4)
#     print(f"Drugs data successfully saved to {output_file}")

# # 示意用法

# convert_drugs_to_json(sel_data_noInt, sel_drugs_names, "sel_drugs_NoInteractions.json")


Drugs data successfully saved to sel_drugs_NoInteractions.json


In [10]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path='./drugs_NoInteractions.json',
    jq_schema='.[].drug',
    text_content=False)

data = loader.load()


In [11]:
docs_list = data

In [208]:
# # for sel_drugs_NoInteractions
# for i in range(len(data)):
#     pat1 = r'\\'
#     pat2 = r'drug-name='
#     tmp = re.sub(pat1, '', str(drug_docs_list[i].page_content), flags=re.DOTALL)
#     drug_docs_list[i].page_content = re.sub(pat2, '', tmp, flags=re.DOTALL)

In [12]:
# 找出不在 drug_list 的索引
in_drug_list_indices = [index for index, drug in enumerate(full_drugs_names) if drug in drug_list]

In [13]:
drug_docs_list = []
for index in in_drug_list_indices:
    drug_docs_list.append(docs_list[index])

In [14]:
for i in range(len(drug_docs_list)):
    drug_docs_list[i].metadata['drug_name'] = sel_drugs_names[i]

In [15]:
print(drug_docs_list[0].metadata)

{'source': '/datasets/soc-20241121132750/Hackathon/VectorStore/drugs_NoInteractions.json', 'seq_num': 5, 'drug_name': 'Etanercept'}


In [16]:
import time

split_start_time = time.time()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
) 
# chunk_size=250

# 使用200個藥物當作輸入
print(len(drug_docs_list))
doc_splits = text_splitter.split_documents(drug_docs_list)

split_end_time = time.time()

474


In [17]:
print("splitting words took:", split_end_time - split_start_time, "seconds")

splitting words took: 107.91893744468689 seconds


In [18]:
doc_splits[100].metadata

{'source': '/datasets/soc-20241121132750/Hackathon/VectorStore/drugs_NoInteractions.json',
 'seq_num': 34,
 'drug_name': 'Desmopressin'}

## Embedding 建立

In [257]:



# ------------------------
# Embedding 到 Vector Store 計時
# ------------------------
persist_directory = "./chroma_langchain_db"

# 初始化向量資料庫（如果需要重新使用已存儲的資料）
# vectorstore = Chroma(
#     # Drug_Single_Split-chroma
#     collection_name="Drug_Single_Split-chroma_final",
#     embedding_function=GPT4AllEmbeddings(model_name=model_name, gpt4all_kwargs=gpt4all_kwargs),
#     persist_directory=persist_directory
# )

embed_start_time = time.time()

# 使用文件新增向量資料庫
vectorstore = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="Drug_Single_Split-chroma_final",
#     embedding=GPT4AllEmbeddings(model_name=model_name, gpt4all_kwargs=gpt4all_kwargs),
#     persist_directory=persist_directory
# )

embed_end_time = time.time()
print("Embedding to Vector Store took:", embed_end_time - embed_start_time, "seconds")

# 獲取檢索器
retriever = vectorstore.as_retriever()

# 持久化數據
vectorstore.persist()




Embedding to Vector Store took: 8.828888654708862 seconds


/tmp/ipykernel_203/4010791135.py:30: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
# Test the retriever
persist_directory="./chroma_langchain_db"

model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs,
)

## "Drug_Single_Split-chroma_final" : 89bba0e9-6791-4aa1-9513-34dc7e9d697d
collection_name = "Drug_Single_Split-chroma_final"

# 重新載入向量資料庫（不用再次加入文件及embed，會直接使用已持久化的檔案）
vectorstore = Chroma(
    collection_name=collection_name,
    embedding_function=GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs,
    ),
    persist_directory=persist_directory
)

# 現在可直接使用 vectorstore 的檢索功能
retriever = vectorstore.as_retriever()

query = "Drug {Lipitor} Tag {Food Interactions}"


# results = retriever.invoke(query)
# print(len(results))
# print(results[1])



In [80]:
# tag = "pubmed-id, general, reference, paper, articles, authors, citations, ref-id, scientfic"
tag = "products"
# tag = "articles"
drug = "Infliximab"

start_time = time.time()

query = f"\"{tag}: information\""
results = vectorstore.similarity_search(query, filter={"drug_name": f"{drug}"})

end_time = time.time()
print("Retrieve from Vector Store took:", end_time - start_time, "seconds")

In [81]:
results[0]

Document(metadata={'drug_name': 'Infliximab', 'seq_num': 62, 'source': '/mnt/dataset-soc-20241121132750/Hackathon/VectorStore/drugs_NoInteractions.json'}, page_content='"over-the-counter": "false", "approved": "true", "country": "EU", "source": "EMA"}, {"name": "Inflectra", "labeller": "Pfizer Europe Ma Eeig", "ndc-id": "", "ndc-product-code": "", "dpd-id": "", "ema-product-code": "EMEA/H/C/002778", "ema-ma-number": "EU/1/13/854/002", "started-marketing-on": "2016-09-08", "ended-marketing-on": "", "dosage-form": "Injection, powder, for solution", "strength": "100 mg", "route": "Intravenous", "fda-application-number": "", "generic": "false", "over-the-counter": "false", "approved": "true", "country": "EU", "source": "EMA"}, {"name": "Inflectra", "labeller": "Pfizer Europe Ma Eeig", "ndc-id": "", "ndc-product-code": "", "dpd-id": "", "ema-product-code": "EMEA/H/C/002778", "ema-ma-number": "EU/1/13/854/003", "started-marketing-on": "2016-09-08", "ended-marketing-on": "", "dosage-form": "I

In [10]:
# import pandas as pd
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.docstore.document import Document
# from langchain_openai import OpenAIEmbeddings
# # Test the retriever
# persist_directory="./chroma_langchain_db"

# model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
# gpt4all_kwargs = {'allow_download': 'True'}
# GPT4AllEmbeddings(
#     model_name=model_name,
#     gpt4all_kwargs=gpt4all_kwargs,
# )
# collection_name = "Drug_Single_Split-chroma"

# f5a330e9-6aea-45b3-938b-13053ca30bbd

# # 重新載入向量資料庫（不用再次加入文件及embed，會直接使用已持久化的檔案）
# vectorstore = Chroma(
#     collection_name=collection_name,
#     embedding_function=GPT4AllEmbeddings(
#     model_name=model_name,
#     gpt4all_kwargs=gpt4all_kwargs,
#     ),
#     persist_directory=persist_directory
# )

# # 現在可直接使用 vectorstore 的檢索功能
# retriever = vectorstore.as_retriever()

# query = "Etanercept"


# results = retriever.invoke(query)
# print(len(results))
# print(results[0])


4
page_content='solution", "strength": "2.5 mg/1mL", "route": "Epidural; Infiltration", "fda-application-number": "", "generic": "false", "over-the-counter": "false", "approved": "true", "country": "US", "source": "FDA NDC"}, {"name": "Bupivacaine Hydrochloride", "labeller": "Hospira, Inc.", "ndc-id": "", "ndc-product-code": "0409-4274", "dpd-id": "", "ema-product-code": "", "ema-ma-number": "", "started-marketing-on": "2007-03-31", "ended-marketing-on": "2007-05-31", "dosage-form": "Injection, solution", "strength": "7.5 mg/1mL", "route": "Epidural; Retrobulbar", "fda-application-number": "", "generic": "false", "over-the-counter": "false", "approved": "true", "country": "US", "source": "FDA NDC"}, {"name": "Bupivacaine Hydrochloride", "labeller": "Hospira, Inc.", "ndc-id": "", "ndc-product-code": "0409-1164", "dpd-id": "", "ema-product-code": "", "ema-ma-number": "", "started-marketing-on": "2007-03-31", "ended-marketing-on": "2007-05-31", "dosage-form": "Injection, solution", "stren